<a href="https://colab.research.google.com/github/AndyCoder2023/CE4145-NLP-Coursework/blob/main/CE4145_Coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

“Generative AI was used to support completion of this assessment. If used: The GenAI tool [Gemini] was used for the purpose of [ideation/editing]. Where used for the purpose of supporting development,
comments have been provided against relevant cells.

Prompt [1]:
Please explain this error:
The error FileNotFoundError: hf://datasets/Pablinho/movies-dataset/9000plus.csv not found in cell 8Ypu2LZD3FOP means that the code is looking for a file at the specified Hugging Face dataset path, but it cannot find it.
Looking at the available variables, it seems the movie data is already loaded into the movie_df DataFrame. I will remove the redundant data loading code from cell 8Ypu2LZD3FOP and use the existing movie_df to create the x and y variables for the NLP pipeline.
It seems there are missing values in the data that are causing the ValueError: Input contains NaN. I will add code to remove rows with missing values in the 'Overview' or 'Genre' columns before processing the data.

Prompt [2]
"ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'numpy.ndarray": The error in cell Ri_Mi50-KjQh is a ValueError because the spaCy nlp function expects a single document (a string, Doc, or bytes), but it received a NumPy array containing all the overviews. To fix this, I will iterate through each overview in the 'Overview' column of the df DataFrame and apply the nlp function to each one individually. I will also add code to skip any missing values in the 'Overview' column to prevent further errors.

**Describe the dataset and the task to be performed**

# most occuring genre
# list all the superheroes

# Dataset

The dataset has 9 columns with 9.84k rows of movie data. The task that is to be performed for the dataset will consist of two different pipelines which will compare the best strategy for the given task to be performed on the movie dataset. The main problem solving task at hand will to find out what the  superhero names are using the statistics available in the dataset. An NLP system is required for this objective to find and narrow down the dataset to pin point the superheros that occur in the dataset. By implementing an NLP system there can be strong analysis on the dataset to find the correct criteria.

** Debating whether to use this: Merchán-Rivera, P. (2025). movies-dataset [Dataset]. Hugging Face. https://huggingface.co/datasets/Pablinho/movies-dataset.

Reference: https://colab.research.google.com/drive/1MbiTzuPPCKrj9BOu_Nj4gcsxxYKwLg0a?usp=sharing#scrollTo=rubk2y4rGx7y


In [6]:
from google.colab import files #import the files package from google.colab framework to be able to upload files

from sklearn.pipeline import Pipeline #let's import the pipeline functionality
from sklearn.feature_extraction.text import CountVectorizer #and we will import a simple pre-processing method
from sklearn.feature_extraction.text import TfidfTransformer #and a representation learner
from sklearn.neighbors import KNeighborsClassifier #and a simple classifier model
from sklearn.model_selection import StratifiedKFold #cross fold is sometimes called k-fold. Calling the stratified version ensures that classes have equal representation across folds
from sklearn.metrics import accuracy_score #import an accuracy metric to tell us how well the model is doing

import numpy as np
np.random.seed(1337)

import pandas as pd

df = pd.read_csv("hf://datasets/Pablinho/movies-dataset/9000plus.csv")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
# Used ai to fix gathering dataset not being found. [1]

x = df['Title'].astype(str).values
y = df['Genre'].astype(str).values

text_clf = Pipeline([ #the pipeline object allows us to organise a series of functions which will be applied to our text data as though they were a single function
  ('prep', CountVectorizer()), #we will use a simple count vectorizer for our pre-processing (which cheats a little by combining numerous pre-processing steps)
  ('rep', TfidfTransformer()), #and a representation learning method using tf-idf
  ('mod', KNeighborsClassifier()), #and a simple kNN classifier
  ])

acc_score = [] #create a list to store the accuracy values

kf = StratifiedKFold(n_splits=5) #we instantiate the kfold instance, and set the number of folds to 5
for train, test in kf.split(x,y): #we use a for loop to iterate through each fold using the train and test indexes from the dataset

  x_train, x_test, y_train, y_test = x[train], x[test], y[train], y[test] #things can get a bit weird when inputting indexes to functions, so lets save them as variables
  #print(train)
  #print(test) #this will print the train and test indexes respectively, if you want to be sure they do not overlap

  text_clf.fit(x_train, y_train) #we then only fit the training data (note that we oapply the text_clf pipeline object, rather than having to go through each function separately)
  predictions = text_clf.predict(x_test) #and can predict on the test data (similar to above, we can predict using the pipeline directly)
  acc = accuracy_score(predictions, y_test) #we use the accuracy score we imported to give an idea how well the model is doing
  acc_score.append(acc) #we can append it to our list

print("Accuracy:", np.mean(acc_score))

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Accuracy: 0.02886933177923543


In [8]:
print(x_train[0]) #first let's double check our input data is just a simple sentence

Spider-Man: No Way Home


In [9]:
import nltk #import the natural language toolkit

nltk.download('punkt') #download the package in nltk which supports tokenization
nltk.download('stopwords') #download the nltk package for stopwords

from nltk.tokenize import word_tokenize #import the tokenize package
from nltk.corpus import stopwords #import the package from the corpus
from nltk.stem.snowball import SnowballStemmer #import the snowball stemmer (also known as Porter2)

from sklearn.base import BaseEstimator, TransformerMixin

class pre_process(BaseEstimator, TransformerMixin):

    def __init__(self):
      return None #we do not need any parameters to instantiate this class

    def fit(self, X, y=None): #both fit and transform expect the data instances and labels to be called - we do not use the labels, so set y=None
        return self #as explained above, we will not use the fit method

    def transform(self, X, y=None):
      prep_text = []
      for x in X: #for each sentence in the whole dataset
            token_text = word_tokenize(x) #tokenize the document
            normd_text = [token.lower() for token in token_text if token.isalpha()] #list compression to apply some simple cleaning (lower case casting and punctuation removal) to tokenized terms

            swr_text = [token for token in normd_text if token not in stopwords.words('english')] #list compression to remove any stopwords from our list

            stemmer = SnowballStemmer("english") #specify we are using the English stemming rules, as other languages are present in toolkit
            prep_text += [[stemmer.stem(word) for word in swr_text]] #list compression for applying the stemmer

      prep_sentences = [" ".join(sentence) for sentence in prep_text] #we join the sentences back together to ensure compatibility with CountVec, which is doing some of it's own prep
      return prep_sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Representation Learning

The reprensentation learning method used will be the Word-2-Vector(Word2Vec). Word2Vec is a technique which is in Natural language processing. The technique permits words to have reprentation as vectors which are in a continuous vector space. The representation learned for every word will be "prediciting a target term using its context(CBOW)"(Martin, 2025, slide 49). Continous Bag of Words (Continuos Bag of Words) model will be used to predict a word that is given words that is located in a window. Input layer will have the context words while output layer will have a current word. Hidden layer has dimensions that will show a current word that is located on the output layer.(Kadam, 2025)


Kyle, M.(2025)'Natural Language Processing'[PowerPoint presentation]. CE4145: CE4145 & CEM300 W5 - Text Pre-Processing & Representation Learning. Available at: https://campusmoodle.rgu.ac.uk/mod/resource/view.php?id=5411825 (Accessed: 18 October 2025).

Sumedh, K.(2025) Word Embedding using Word2Vec. Available at: https://www.geeksforgeeks.org/python/python-word-embedding-using-word2vec/ (Accessed: 22 October 2025).

Manan S.(2022) A Dummy's Guide to Word2Vec. Available at: https://medium.com/@manansuri/a-dummys-guide-to-word2vec-456444f3c673 (Accessed: 01 November 2025).

In [10]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 63.7 MB/s eta 0:00:00


In [11]:
from gensim.models import Word2Vec #first we import word2vec, then a few familiar imports

In [12]:
# Assigning string values
texts = df['Overview'].astype(str).values

In [13]:
sentences = [line.split() for line in texts]

w2v = Word2Vec(sentences, window=5, workers=4)

In [ ]:
print(sentences[0])

['Peter', 'Parker', 'is', 'unmasked', 'and', 'no', 'longer', 'able', 'to', 'separate', 'his', 'normal', 'life', 'from', 'the', 'high-stakes', 'of', 'being', 'a', 'super-hero.', 'When', 'he', 'asks', 'for', 'help', 'from', 'Doctor', 'Strange', 'the', 'stakes', 'become', 'even', 'more', 'dangerous,', 'forcing', 'him', 'to', 'discover', 'what', 'it', 'truly', 'means', 'to', 'be', 'Spider-Man.']


Describe the theory behind the
algorithms to be applied. **bold text**

# Algorithms

Multi-Layer Perceptron(MLP) is able to learn a non - linear function that is used to seperate data. All the layers in a multi layer perceptron are closely connected. Multiple hidden layers are also possible. Multilayer perceptrons have an input layer, hidden layer, and an output layer. For the input layer there needs to be a numerical vector. The number of indexes in a vector will be the size of input layer. Hidden layer is used to connect input and output layer by using weights & biases. There is an activation function in the hidden layer but it is usually different on each hidden layer. The output layer is dependant on the final activation function but normally softmax is used in classification. The ouput layer consists of a vector of probabilities that have the same length of the number of classes.

Named Entity recognition(NER) works by grabbing the list of named entities that are in a document. The algorithm then labels the named entities into categories which consist of "person names, organizations, locations, medical codes, time expressions, quantities, monetary values, and more."(Awan,2023). NER works by connecting unstructured and structured data. This allows a computer to go through lots of data and grab the necessary data that is highly valued.


Abid A.(2023) What is Named Entity Recognition (NER)? Methods, Use Cases, and Challenges. Available at: https://www.datacamp.com/blog/what-is-named-entity-recognition-ner
(Accessed: 01 November 2025).


In [14]:
!pip install spacy
!pip install nltk
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 51.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [19]:
print(df) #if we print the dataset, we will see it appears as a dictionary
print(df['Overview'][0])

     Release_Date                                 Title  \
0      2021-12-15               Spider-Man: No Way Home   
1      2022-03-01                            The Batman   
2      2022-02-25                               No Exit   
3      2021-11-24                               Encanto   
4      2021-12-22                        The King's Man   
...           ...                                   ...   
9832   1973-10-15                              Badlands   
9833   2020-10-01                      Violent Delights   
9834   2016-05-06                          The Offering   
9835   2021-03-31  The United States vs. Billie Holiday   
9836   1984-09-23                               Threads   

                                               Overview  Popularity  \
0     Peter Parker is unmasked and no longer able to...    5083.954   
1     In his second year of fighting crime, Batman u...    3827.658   
2     Stranded at a rest stop in the mountains durin...    2618.087   
3     T

In [15]:
import pandas as pd
import spacy

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# View columns
print(df.columns)


Index(['Release_Date', 'Title', 'Overview', 'Popularity', 'Vote_Count',
       'Vote_Average', 'Original_Language', 'Genre', 'Poster_Url'],
      dtype='object')


In [ ]:

#Asked AI for support with gathering the entity names [2]

# Drop rows with missing values in 'Overview' column before processing
df.dropna(subset=['Title'], inplace=True)

# Iterate through the 'Overview' column and apply nlp to each overview
for title in df['Title']:
    doc = nlp(title)
    for ent in doc.ents:
        print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [17]:
from spacy import displacy
displacy.render(doc, style="ent")

In [4]:
entities = [(ent.text, ent.label_, ent.lemma_) for ent in doc.ents]
df = pd.DataFrame(entities, columns=['text', 'type', 'lemma'])
print(df)

NameError: name 'doc' is not defined

In [3]:
from spacy.training.example import Example #import the data structure required as input

train_data = [
    ("Tom 0 3", {"entities":[(0,4, "PERSON")]}), #create some training data, which is annotated text
    ("This is an example of Tom 0 3", {"entities":[(13,16, "Algorithms")]}) #note we state the start and end character and a new label
    ]

ner=df.get_pipe("ner") #we then extract specifically the ner component of the pipeline

for n in range (100): #training will take multiple iterations
  for text, annotations in train_data:
    for ent in annotations.get("entities"): #we extract the entity component
      ner.add_label(ent[2]) #and add the label to our list of possible entities
    # create Example
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotations) #we then put the information into an Example data structure
    # Update the model
    nlp.update([example]) #and update the model

NameError: name 'df' is not defined

Reference to add: https://www.geeksforgeeks.org/nlp/named-entity-recognition/

# MLP Output



In [28]:
from tensorflow import keras #we will only be using tensorflow as a backend, so we will import keras via their implementation
from keras import layers #we will use the readable keras implementation of the dense layers
from keras import activations #and the keras activation functions


In [36]:
mlp = keras.Sequential( #first we create a model and specify we are using the Sequential configuration within Keras
    [
    keras.Input(shape=(95,)), #Next we specify the size of our input layer. For our dataset we have 95 features. If the data were multiple dimensions, we would add more after the comma.
    keras.layers.Dense(64, activations.relu), #Then we have our hidden layer. We have used 64 neurons, with a relu activation (consistent with above)
    keras.layers.Dense(2, activations.softmax) #We then send to our output layer (which has 2 neurons, one for each class) and a softmax activation function
    ]
)

mlp.summary() #we can view a summary of the model, including the number of trainable parameters (i.e. weights + biases) and the storage size.

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 64)             │         6,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,274 (24.51 KB)

 Trainable params: 6,274 (24.51 KB)

 Non-trainable params: 0 (0.00 B)

In [39]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=1337) #let's use a static split for clarity

num_classes = 2 #we need to identify the number of output classes - here we have 2 - bankrupt or not.

#Then we convert class vectors to one hot vectors - i.e. vectors of the size of number of classes, with a 1 in the class index and a 0 in every other class index
y_traink = keras.utils.to_categorical(y_train, num_classes)
y_testk = keras.utils.to_categorical(y_test, num_classes)

batch_size = 64 #the batch size is the number of examples the model will view before summing and backpropagating the loss
epochs = 10 #epochs are the number of training iterations examining the full training set
loss = "categorical_crossentropy" #loss is how we measure the error. Categorical crossentropy compares the probability for a prediction with it's ground truth (i.e. the one-hot vector we just created above)
optimizer="sgd" #optimizer updates the parameters on the backward pass. SGD is a type of gradient descent to do this.


mlp.compile(loss=loss, optimizer=optimizer, metrics=["accuracy", "f1_score"]) #We can set the model to compile.
mlp.fit(x_train, y_traink, batch_size=batch_size, epochs=epochs) #Finally, we train the model


ValueError: invalid literal for int() with base 10: 'Horror, Comedy'

In [38]:
#We can then test our model on the test set
score = mlp.evaluate(x_test, y_testk) #keras has an inbuilt evaluation function that will produce predictions and scores for a test set.

NameError: name 'y_testk' is not defined

Describe the evaluation process, and
analyse the results. **bold text**

# Evaluation

In [ ]:
import matplotlib.pyplot as plt

scores = [0.4826, 0.4718, 0.4579, 0.4579, 0.4764]
alg_scores = [f1_scores] + [scores]

fig, ax = plt.subplots()
ax.set_title('Algorithm Performance')
ax.boxplot(alg_scores)
ax.set_xticklabels(["SKLearn", "Keras"], rotation='vertical')